<a href="https://colab.research.google.com/github/jtao22/PythonAI/blob/main/Astrophysics/exoplanetIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#import 
import numpy as np
import pandas as pd 
from google.colab import files 
from keras.models import Sequential as seq
from keras.layers import Dense, Activation,Dropout, Conv1D, Flatten, Reshape
from keras.layers import MaxPooling1D as mp1d
from keras.layers import GlobalMaxPooling1D as gmp1d
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix as CM
from sklearn.metrics import accuracy_score as acc
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
files.upload()

Saving exoTest.csv to exoTest (1).csv
Saving exoTrain.csv to exoTrain.csv


In [26]:
train = pd.read_csv('exoTrain.csv')
test = pd.read_csv('exoTest.csv')
print(train.shape)
print(test.shape)

(5087, 3198)
(570, 3198)


In [28]:
train.head(5)

LABEL   FLUX.1   FLUX.2   FLUX.3  ...  FLUX.3194  FLUX.3195  FLUX.3196  FLUX.3197
0      2    93.85    83.81    20.10  ...      39.32      61.42       5.08     -39.54
1      2   -38.88   -33.83   -58.54  ...     -11.70       6.46      16.00      19.93
2      2   532.64   535.92   513.73  ...     -11.80     -28.91     -70.02     -96.67
3      2   326.52   347.39   302.35  ...      -8.77     -17.31     -17.35      13.98
4      2 -1107.21 -1112.59 -1118.95  ...    -399.71    -384.65    -411.79    -510.54

[5 rows x 3198 columns]

In [30]:
testY = test['LABEL']
trainY = train['LABEL']
test = test.drop(columns=['LABEL'],axis=1)
train = train.drop(columns=['LABEL'], axis=1)

In [37]:
#visualize
fig = plt.figure(figsize=(15,15))
plot1 = fig.add_subplot(3,1,1)
plot2 = fig.add_subplot(3,1,2)
plot3 = fig.add_subplot(3,1,3)
plot1.plot([int(i.replace('FLUX.', '')) for i in train.iloc[0].index], train.iloc[0])
plot1.set_xlabel('t')
plot1.set_ylabel('flux')
plot2.plot([int(i.replace('FLUX.', '')) for i in train.iloc[1].index], train.iloc[1])
plot2.set_xlabel('t')
plot2.set_ylabel('flux')
plot3.plot([int(i.replace('FLUX.', '')) for i in train.iloc[2].index], train.iloc[2])
plot3.set_xlabel('t')
plot3.set_ylabel('flux')
fig.show()

In [38]:
train.head(5)

FLUX.1   FLUX.2   FLUX.3  ...  FLUX.3195  FLUX.3196  FLUX.3197
0    93.85    83.81    20.10  ...      61.42       5.08     -39.54
1   -38.88   -33.83   -58.54  ...       6.46      16.00      19.93
2   532.64   535.92   513.73  ...     -28.91     -70.02     -96.67
3   326.52   347.39   302.35  ...     -17.31     -17.35      13.98
4 -1107.21 -1112.59 -1118.95  ...    -384.65    -411.79    -510.54

[5 rows x 3197 columns]

In [39]:
mms = MMS()
train = mms.fit_transform(train)
test = mms.fit_transform(test)
le = LE()
testY = le.fit_transform(testY)
trainY = le.fit_transform(trainY)

In [52]:
print(train.shape[1])

3197


In [53]:
#create 1D Convolutional NN
arch = seq()
arch.add(Reshape((train.shape[1], 1), input_shape=(train.shape[1],)))
arch.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(train.shape[1], 1)))
arch.add(mp1d(pool_size=8))
arch.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
arch.add(mp1d(pool_size=8))
arch.add(Flatten())
arch.add(Dense(512, activation='relu'))
arch.add(Dropout(0.5)) 
arch.add(Dense(512, activation='relu')) 
arch.add(Dropout(0.5)) 
arch.add(Dense(64, activation='relu'))
arch.add(Dropout(0.5)) 
arch.add(Dense(32, activation='relu'))
arch.add(Dense(1))
arch.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['acc'])

In [54]:
arch.fit(train,trainY, batch_size=64, epochs=4, validation_split = 0.3, shuffle=True)

Epoch 1/4
56/56 [==============================] - 8s 130ms/step - loss: 0.1737 - acc: 0.9879 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/4
56/56 [==============================] - 7s 125ms/step - loss: 0.1286 - acc: 0.9917 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/4
56/56 [==============================] - 7s 126ms/step - loss: 0.1461 - acc: 0.9905 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/4
56/56 [==============================] - 7s 126ms/step - loss: 0.1885 - acc: 0.9878 - val_loss: 0.0000e+00 - val_acc: 1.0000


In [64]:
predictions = arch.predict_classes(test)
print(acc(predictions,testY)*100,'%')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


99.12280701754386 %
